In [1]:
setwd('/import/DB/fel/BJTC-204')
library(Seurat)
sce = readRDS('male.sc.rds')

Warning message:
"package 'Seurat' was built under R version 4.2.3"
The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject



In [5]:
options(future.globals.maxSize= 1000*1024^2)

In [6]:
Idents(sce) = 'celltype'

In [11]:
sce$group = sce$Condition

In [12]:
table(sce$group)


   Case Control 
  42914   35065 

In [14]:
ExN = subset(sce,idents = 'ExN')

In [32]:
DefaultAssay(ExN) = 'RNA'

In [33]:
Idents(ExN)='group'
deg=FindMarkers(object = ExN, ident.1 = 'Case',ident.2 = 'Control',
 min.pct = 0.01, logfc.threshold = 0.01,
 thresh.use = 0.99)
head(deg)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ACTB,0.000000e+00,-0.2565893,0.582,0.733,0.000000e+00
GAPDH,0.000000e+00,-0.3198014,0.428,0.625,0.000000e+00
XIST,0.000000e+00,-0.2250103,0.001,0.078,0.000000e+00
PHYHIP,1.850210e-293,-0.2598557,0.425,0.585,6.209676e-289
CKB,1.127514e-290,-0.2343880,0.583,0.730,3.784163e-286
TUBB4A,3.380432e-290,-0.2439137,0.177,0.323,1.134541e-285


In [34]:
diffgene = deg

In [35]:
library(clusterProfiler)
library(enrichplot)
library(org.Mm.eg.db)
#为每个基因添加对应的ENTREZID
diffgene$gene <- rownames(diffgene)
ids=bitr(diffgene$gene,'SYMBOL','ENTREZID','org.Hs.eg.db')
#合并数据，cluser3.markers中没有ENTREZID的基因将被过虑掉
diffgene=merge(diffgene,ids,by.x='gene',by.y='SYMBOL')

head(diffgene)

cluster3.markers = diffgene

#将基因按照avg_log2FC的大小进行降序排列
cluster3.markers <- cluster3.markers[order(cluster3.markers$avg_log2FC,decreasing = T),]
#生成仅含有ENTREZID名字和avg_log2FC值的gene list
cluster3.markers_list <- as.numeric(cluster3.markers$avg_log2FC)
names(cluster3.markers_list) <- cluster3.markers$ENTREZID
head(cluster3.markers_list)

#筛选差异较大的基因集
cluster3_de <- names(cluster3.markers_list)[abs(cluster3.markers_list) > 0]
head(cluster3_de)

length(cluster3_de)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(diffgene$gene, "SYMBOL", "ENTREZID", "org.Hs.eg.db"):
"8.71% of input gene IDs are fail to map..."


,gene,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,ENTREZID
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,A1BG,2.130260e-01,0.01976327,0.071,0.069,1.000000e+00,1
2,A2ML1-AS1,6.746000e-02,0.02582027,0.117,0.114,1.000000e+00,100874108
3,AAAS,4.370983e-01,0.01553439,0.095,0.094,1.000000e+00,8086
4,AACS,3.746303e-07,-0.01626441,0.261,0.292,1.257334e-02,65985
5,AAMP,6.617257e-14,-0.02004674,0.100,0.124,2.220884e-09,14
6,AAR2,1.106243e-11,-0.01434626,0.033,0.046,3.712772e-07,25980


6129      6144 100506860      6233     57035     23037 
0.2070436 0.1828031 0.1764819 0.1516022 0.1390246 0.1257692

[1] "6129"      "6144"      "100506860" "6233"      "57035"     "23037"

[1] 8043

In [36]:
cluster3_gsekg <- gseKEGG(cluster3.markers_list,organism = "hsa",pvalueCutoff = 0.05)
#将富集结果按照NES绝对值降序排列
cluster3_gsekg_arrange <- arrange(cluster3_gsekg,desc(abs(NES)))

preparing geneSet collections...

GSEA analysis...

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
"There were 3 pathways for which P-values were not calculated properly due to unbalanced (positive and negative) gene-level statistic values. For such pathways pval, padj, NES, log2err are set to NA. You can try to increase the value of the argument nPermSimple (for example set it nPermSimple = 10000)"
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
"For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation."
leading edge analysis...

done...



In [37]:
cluster3_gsekg_arrange@result

,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
hsa00190,hsa00190,Oxidative phosphorylation,97,-0.7241892,-2.481337,1.000000e-10,2.121429e-09,1.345865e-09,1242,"tags=69%, list=15%, signal=59%",4701/6389/5464/4695/55967/521/4700/10476/1347/509/27089/4729/4723/1351/4717/534/1349/51382/4696/522/54539/4694/4714/4702/514/4725/54205/4728/518/1346/9377/498/29796/4718/374291/4710/517/1350/1537/126328/1327/1345/9551/4713/516/4722/7384/7388/51079/4716/9114/529/535/9550/537/1329/4720/526/1337/1340/506/155066/4697/533/9296/513/527
hsa04145,hsa04145,Phagosome,66,-0.7393893,-2.452129,1.000000e-10,2.121429e-09,1.345865e-09,714,"tags=44%, list=9%, signal=40%",534/51382/5869/5878/3916/811/821/1778/9114/529/535/9550/537/526/5879/155066/71/533/9296/7846/10383/7277/203068/10381/7280/527/10376/10382/60
hsa04721,hsa04721,Synaptic vesicle cycle,50,-0.7490616,-2.425059,1.000000e-10,2.121429e-09,1.345865e-09,843,"tags=58%, list=10%, signal=52%",161/160/534/51382/163/1211/1759/4905/6812/6506/9114/529/535/9550/10815/1175/10814/112755/1212/526/8775/155066/6844/533/57030/9296/1173/5864/527
hsa05012,hsa05012,Parkinson disease,196,-0.6518474,-2.349269,1.000000e-10,2.121429e-09,1.345865e-09,1282,"tags=56%, list=16%, signal=48%",2771/5694/3309/4701/9927/6389/5717/10131/4695/118424/5693/7345/55967/5719/7419/25828/4700/57181/10476/1347/5691/5708/509/292/27089/4729/7332/5705/4723/6622/1351/5702/4717/7295/1349/5718/5688/5700/5692/5714/5566/4696/522/54539/4694/4714/4702/514/7317/5709/11047/11315/468/815/7416/4725/54205/291/4728/64837/518/1346/9377/498/29796/4718/805/374291/4710/517/1350/1537/126328/1327/1345/4713/516/65018/4722/7384/801/816/7388/293/51079/4716/3798/1329/4720/7311/1337/1340/6647/506/3831/808/4697/7314/513/7846/2778/10383/7316/7277/203068/10381/7280/10376/10382
hsa05020,hsa05020,Prion disease,186,-0.6432076,-2.312638,1.000000e-10,2.121429e-09,1.345865e-09,1460,"tags=57%, list=18%, signal=48%",515/5601/5704/7417/5710/572/5694/3309/4701/6389/3303/5717/4695/5693/3304/55967/5719/7419/1460/4700/10476/10963/1347/5691/5708/509/292/5595/27089/4729/3312/5705/2902/4723/1351/5702/4717/1349/5718/5688/5700/5692/5714/5566/4696/522/54539/4694/4714/4702/514/5709/5295/11047/468/7416/5534/4725/54205/291/4728/64837/518/1346/9377/498/1958/29796/4718/374291/4710/5594/517/1350/1537/126328/1327/1345/4713/516/4722/7384/7388/293/51079/4716/3798/1329/4720/1337/5621/5879/1340/6647/506/3831/4697/513/7846/10383/7277/203068/10381/7280/10376/10382
hsa00010,hsa00010,Glycolysis / Gluconeogenesis,32,-0.7396920,-2.259522,4.776472e-09,8.866325e-08,5.624924e-08,530,"tags=41%, list=7%, signal=38%",5230/3939/5214/3098/3945/2026/5223/2821/2023/230/7167/5315/2597
hsa05415,hsa05415,Diabetic cardiomyopathy,133,-0.6326872,-2.226635,1.000000e-10,2.121429e-09,1.345865e-09,1025,"tags=56%, list=13%, signal=49%",818/5164/515/5162/5601/7417/4701/5582/6389/4695/55967/7419/4700/10476/1347/25874/509/292/27089/4729/4723/1351/4717/1349/4696/522/54539/4694/4714/4702/514/5295/815/7416/5500/4725/291/4728/518/1346/488/9377/498/29796/4718/374291/4710/5590/517/1350/1537/126328/1327/1345/4713/5524/516/4722/7384/816/7388/293/51079/4716/1329/4720/1337/5879/1340/506/1509/4697/513/2597
hsa05016,hsa05016,Huntington disease,216,-0.6127433,-2.220490,1.000000e-10,2.121429e-09,1.345865e-09,1460,"tags=53%, list=18%, signal=45%",515/5601/3066/5704/7417/5710/6648/9776/1213/4294/5694/6507/4701/6389/11258/5717/4695/5438/5609/5693/10120/55967/5719/6804/7419/4700/10476/5434/1347/5691/5708/509/292/161/27089/160/4729/5705/2878/2902/4723/1351/5702/4717/1349/5718/5688/5700/163/5692/5714/4696/522/54539/4694/4714/4702/514/5709/11047/26100/7416/10121/4725/54205/1211/291/4728/64837/518/1346/9377/498/29796/4718/374291/4710/517/1350/1537/126328/1327/1345/4713/516/4722/7384/7388/293/51079/4716/6506/3798/1329/4720/1742/1639/1175/1212/1337/1340/6647/506/3831/4697/513/7846/10383/1173/7277/203068/10381/7280/10376/10382
hsa